## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-01-07-30 +0000,bbc,Trump threatens Brazil with 50% tariff and dem...,https://www.bbc.com/news/articles/c784ee81y4zo
1,2025-07-10-01-05-22 +0000,nyt,Rubio Visits Asia as Trump Raises Trade-War Te...,https://www.nytimes.com/2025/07/09/world/asia/...
2,2025-07-10-01-01-00 +0000,wsj,Bank of Korea Holds Rates as Growth Concerns Ebb,https://www.wsj.com/articles/bank-of-korea-hol...
3,2025-07-10-01-00-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...
4,2025-07-10-00-36-34 +0000,bbc,"US sanctions UN expert Francesca Albanese, cri...",https://www.bbc.com/news/articles/c70rllxr0kyo


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2307/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,27
29,president,6
252,new,5
173,he,5
30,tariffs,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
90,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...,57
3,2025-07-10-01-00-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...,54
46,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,53
64,2025-07-09-12-38-45 +0000,nypost,Trump boasted to donors that he threatened to ...,https://nypost.com/2025/07/09/us-news/trump-bo...,50
22,2025-07-09-20-37-43 +0000,nypost,Mike Johnson steps up calls for fresh Russia s...,https://nypost.com/2025/07/09/us-news/mike-joh...,49


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
90,57,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...
73,25,2025-07-09-10-00-00 +0000,nypost,Secretary of State Marco Rubio gets glowing en...,https://nypost.com/2025/07/09/us-news/secretar...
53,23,2025-07-09-15-06-48 +0000,nypost,Biden doctor Kevin O’Connor invokes Fifth Amen...,https://nypost.com/2025/07/09/us-news/biden-do...
22,22,2025-07-09-20-37-43 +0000,nypost,Mike Johnson steps up calls for fresh Russia s...,https://nypost.com/2025/07/09/us-news/mike-joh...
30,22,2025-07-09-19-48-23 +0000,nyt,Former President Yoon Suk Yeol of South Korea ...,https://www.nytimes.com/2025/07/09/world/asia/...
46,21,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
23,17,2025-07-09-20-36-58 +0000,bbc,'Huge wall of water': Texas man stood on meter...,https://www.bbc.com/news/articles/c3358pz41d6o
77,17,2025-07-09-09-36-00 +0000,wsj,Malaysia’s Central Bank Cuts Rates for First T...,https://www.wsj.com/articles/malaysias-central...
0,16,2025-07-10-01-07-30 +0000,bbc,Trump threatens Brazil with 50% tariff and dem...,https://www.bbc.com/news/articles/c784ee81y4zo
19,15,2025-07-09-21-02-03 +0000,nypost,Ex-Long Island hospital CEO to sue parent comp...,https://nypost.com/2025/07/09/us-news/ousted-l...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
